## Download github repo

In [1]:
#just clone the repo
!git clone https://github.com/nhvd3500111/ditto

In [39]:
#In case you want to reupload ditto if you have made changes in the github
'''
import shutil
shutil.rmtree("/kaggle/working/ditto")
%cd /kaggle/working/
!git clone https://github.com/nhvd3500111/ditto
%cd /kaggle/working/ditto
'''


## Install packages and fp16 optimization

In [5]:
# install requirements
%cd /kaggle/working/ditto
!pip install -r requirements.txt
import nltk
nltk.download('stopwords')
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli
%cd /kaggle/working/ditto
!pip install transformers
!pip install tensorboardX
!pip install jsonlines
!pip install openpyxl

## Train DITTO

In [40]:
# Step 1:
# train the Ditto model_GRU (pre_trained lm=distilbert) with the save_model flag on. 
#It will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.

!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task wdc_cameras_small \
  --batch_size 32 \
  --max_len 128 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm distilbert \
  --save_model \
  --da entry_swap \
  --neural cls_sep_gru


## Running the matcher

In [41]:
# Step 2: run the matcher

#da , dk and summarize must be the same as above trained
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task wdc_cameras_small \
  --input_path data/wdc/cameras/test.txt \
  --output_path output/output_small_1.jsonl \
  --lm distilbert \
  --max_len 128 \
  --use_gpu \
  --da entry_swap\
  --checkpoint_path checkpoints/ \
  --neural cls_sep_gru

#We will download the xlsx file manually from the output folder